In [3]:
from moviepy.editor import VideoFileClip, VideoClip, CompositeVideoClip
import numpy as np

def make_wipe_mask(size, duration):
    """
    Returns a mask clip of given size (w,h) that reveals from
    top to bottom over `duration` seconds.
    """
    w, h = size

    def mask_frame(t):
        # Number of rows to reveal at time t
        rows = int((t / duration) * h)
        mask = np.zeros((h, w), dtype=np.uint8)
        mask[:rows, :] = 255
        return mask

    return VideoClip(make_frame=mask_frame, ismask=True, duration=duration)

def stack_with_wipe_transitions(gif_paths, transition_duration=0.5, output_path="output.gif"):
    # Load all GIFs as clips
    clips = [VideoFileClip(path) for path in gif_paths]

    # Ensure same size (optional; here we'll resize all to the first clip's size)
    target_size = clips[0].size
    for i, clip in enumerate(clips):
        if clip.size != target_size:
            clips[i] = clip.resize(target_size)

    # Start building the composite timeline
    timeline = clips[0]

    # For each next clip, create a wipe mask and overlay it at the end of the previous
    for next_clip in clips[1:]:
        # Create mask animation
        mask = make_wipe_mask(target_size, transition_duration)

        # Position the next clip to start at end of the current timeline
        masked_clip = (
            next_clip
            .set_start(timeline.duration)    # when the wipe should begin
            .set_mask(mask)                 # apply our top-to-bottom mask
        )

        # Composite the current timeline with the incoming masked clip
        timeline = CompositeVideoClip([timeline, masked_clip], size=target_size)

    # Export as GIF (you can also use write_videofile for MP4/WEBM)
    timeline.write_gif(output_path, fps=clips[0].fps)

# if __name__ == "__main__":
#     # Replace these with your actual filenames
#     gifs = ["media/gifs/1.gif", "media/gifs/2.gif", "media/gifs/3.gif"]
#     stack_with_wipe_transitions(gifs,
#                                 transition_duration=0.7,   # seconds per wipe
#                                 output_path="combined.gif")
#     print("Done! Check combined.gif")


In [4]:
# add this before any moviepy imports
from PIL import Image
if not hasattr(Image, "ANTIALIAS"):
    # Pillow ≥10 has moved ANTIALIAS under Resampling
    Image.ANTIALIAS = Image.Resampling.LANCZOS

from moviepy.editor import VideoFileClip, VideoClip, CompositeVideoClip
import numpy as np

def make_wipe_mask(size, duration):
    w, h = size
    def mask_frame(t):
        rows = int((t / duration) * h)
        mask = np.zeros((h, w), dtype=np.uint8)
        mask[:rows, :] = 255
        return mask
    return VideoClip(make_frame=mask_frame, ismask=True, duration=duration)

def stack_with_wipe_transitions(gif_paths, transition_duration=0.5, output_path="output.gif"):
    clips = [VideoFileClip(path) for path in gif_paths]
    target_size = clips[0].size
    for i, clip in enumerate(clips):
        if clip.size != target_size:
            clips[i] = clip.resize(target_size)  # now uses patched Image.ANTIALIAS

    timeline = clips[0]
    for next_clip in clips[1:]:
        mask = make_wipe_mask(target_size, transition_duration)
        masked = next_clip.set_start(timeline.duration).set_mask(mask)
        timeline = CompositeVideoClip([timeline, masked], size=target_size)

    timeline.write_gif(output_path, fps=clips[0].fps)

if __name__ == "__main__":
    gifs = ["media/gifs/1.gif", "media/gifs/2.gif", "media/gifs/3.gif"]
    stack_with_wipe_transitions(
        gifs,
        transition_duration=0.7,
        output_path="combined.gif"
    )
    print("Done! Check combined.gif")


MoviePy - Building file combined.gif with imageio.


Done! Check combined.gif


In [ ]:
# patch Pillow for MoviePy if using Pillow ≥10
from PIL import Image, ImageFilter
if not hasattr(Image, "ANTIALIAS"):
    Image.ANTIALIAS = Image.Resampling.LANCZOS

from moviepy.editor import VideoFileClip, VideoClip, CompositeVideoClip, vfx
import numpy as np

def make_wipe_mask(size, duration):
    w, h = size
    def mask_frame(t):
        rows = int((t / duration) * h)
        mask = np.zeros((h, w), dtype=np.uint8)
        mask[:rows, :] = 255
        return mask
    return VideoClip(make_frame=mask_frame, ismask=True, duration=duration)

def stack_with_wipe_transitions(
    gif_paths,
    transition_duration=0.5,
    speed_factor=2.0,
    output_path="combined.gif"
):
    clips = []
    for i, path in enumerate(gif_paths):
        clip = VideoFileClip(path)
        # 1) speed up
        clip = clip.fx(vfx.speedx, speed_factor)

        # 2) for the 3rd GIF only, apply a tiny median filter per-frame
        if i == 2:
            def denoise_frame(frame):
                # frame is a H×W×3 uint8 ndarray
                pil = Image.fromarray(frame)
                filtered = pil.filter(ImageFilter.MedianFilter(size=3))
                return np.array(filtered)
            clip = clip.fl_image(denoise_frame)

        clips.append(clip)

    # unify sizes
    target_size = clips[0].size
    for i, clip in enumerate(clips):
        if clip.size != target_size:
            clips[i] = clip.resize(target_size)

    # build timeline with top→bottom wipes
    timeline = clips[0]
    for next_clip in clips[1:]:
        mask = make_wipe_mask(target_size, transition_duration)
        incoming = next_clip.set_start(timeline.duration).set_mask(mask)
        timeline = CompositeVideoClip(
            [timeline, incoming],
            size=target_size
        )

    # export
    timeline.write_gif(output_path, fps=clips[0].fps)

if __name__ == "__main__":
    gifs = [
        "media/gifs/2.gif",
        "media/gifs/1.gif",
        "media/gifs/3.gif"
    ]
    stack_with_wipe_transitions(
        gifs,
        transition_duration=0.7,   # seconds per wipe
        speed_factor=2.0,          # 2× speed everywhere
        output_path="media/gifs/combined.gif"
    )
    print("Done! Check combined.gif")


MoviePy - Building file combined.gif with imageio.


Done! Check combined.gif


In [2]:
# patch Pillow for MoviePy if using Pillow ≥10
from PIL import Image, ImageFilter
if not hasattr(Image, "ANTIALIAS"):
    Image.ANTIALIAS = Image.Resampling.LANCZOS

from moviepy.editor import (
    VideoFileClip,
    VideoClip,
    CompositeVideoClip,
    vfx
)
import numpy as np

def make_wipe_mask(size, duration):
    w, h = size
    def mask_frame(t):
        rows = int((t / duration) * h)
        mask = np.zeros((h, w), dtype=np.uint8)
        mask[:rows, :] = 255
        return mask
    return VideoClip(make_frame=mask_frame, ismask=True, duration=duration)

def stack_with_wipe_transitions(
    gif_paths,
    transition_duration=0.5,
    speed_factor=2.0,
    output_path="combined.mp4",
    fps=30
):
    clips = []
    for i, path in enumerate(gif_paths):
        clip = VideoFileClip(path).fx(vfx.speedx, speed_factor)

        # apply light median filter to the 3rd GIF to reduce noise
        if i == 2:
            def denoise_frame(frame):
                pil = Image.fromarray(frame)
                filtered = pil.filter(ImageFilter.MedianFilter(size=3))
                return np.array(filtered)
            clip = clip.fl_image(denoise_frame)

        clips.append(clip)

    # unify sizes
    target_size = clips[0].size
    clips = [
        clip.resize(target_size) if clip.size != target_size else clip
        for clip in clips
    ]

    # build timeline with top→bottom wipes
    timeline = clips[0]
    for next_clip in clips[1:]:
        mask = make_wipe_mask(target_size, transition_duration)
        incoming = next_clip.set_start(timeline.duration).set_mask(mask)
        timeline = CompositeVideoClip([timeline, incoming], size=target_size)

    # export as MP4 for smooth playback
    timeline.write_videofile(
        output_path,
        codec="libx264",
        fps=fps,
        preset="medium",
        threads=4,
        ffmpeg_params=["-pix_fmt", "yuv420p"]
    )

if __name__ == "__main__":
    gifs = [
        "2.gif",
        "1.gif"
    ]
    stack_with_wipe_transitions(
        gifs,
        transition_duration=0.7,   # seconds per wipe
        speed_factor=2.0,          # 2× speed
        output_path="video.mp4",
        fps=30                     # smooth 30 fps output
    )
    print("Done! Check combined.mp4")


Moviepy - Building video video.mp4.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4
Done! Check combined.mp4


In [ ]:
# patch Pillow for MoviePy if using Pillow ≥10
from PIL import Image, ImageFilter
if not hasattr(Image, "ANTIALIAS"):
    Image.ANTIALIAS = Image.Resampling.LANCZOS

from moviepy.editor import (
    VideoFileClip,
    VideoClip,
    CompositeVideoClip,
    ColorClip,
    vfx
)
import numpy as np

def make_wipe_mask(size, duration):
    w, h = size
    def mask_frame(t):
        rows = int((t / duration) * h)
        mask = np.zeros((h, w), dtype=np.uint8)
        mask[:rows, :] = 255
        return mask
    return VideoClip(make_frame=mask_frame, ismask=True, duration=duration)

def stack_with_wipe_transitions(
    gif_paths,
    transition_duration=0.5,
    speed_factor=2.0,
    zoom_out_factor=0.8,
    output_path="combined.mp4",
    fps=30
):
    # 1) Load, speed‐up, zoom‐out & (if 3rd) denoise
    clips = []
    for i, path in enumerate(gif_paths):
        clip = VideoFileClip(path).fx(vfx.speedx, speed_factor)
        clip = clip.fx(vfx.resize, zoom_out_factor)

        # denoise the 3rd GIF lightly
        if i == 2:
            def denoise_frame(frame):
                pil = Image.fromarray(frame)
                filtered = pil.filter(ImageFilter.MedianFilter(size=3))
                return np.array(filtered)
            clip = clip.fl_image(denoise_frame)

        clips.append(clip)

    # 2) Use the first clip’s size as our target canvas
    target_size = clips[0].size

    # 3) Resize/pad each clip
    processed = []
    for idx, clip in enumerate(clips):
        if clip.size != target_size:
            if idx == 1:
                # GIF #2: resize by width, then pad on black background
                clip_resized = clip.resize(width=target_size[0])
                bg = ColorClip(
                    size=target_size,
                    color=(0, 0, 0),
                    duration=clip_resized.duration
                )
                clip = CompositeVideoClip([
                    bg,
                    clip_resized.set_position("center")
                ], size=target_size)
            else:
                # others: force‐resize
                clip = clip.resize(target_size)
        processed.append(clip)
    clips = processed

    # 4) Composite with top→bottom wipes
    timeline = clips[0]
    for next_clip in clips[1:]:
        mask = make_wipe_mask(target_size, transition_duration)
        incoming = next_clip.set_start(timeline.duration).set_mask(mask)
        timeline = CompositeVideoClip([timeline, incoming], size=target_size)

    # 5) Export as smooth MP4
    timeline.write_videofile(
        output_path,
        codec="libx264",
        fps=fps,
        preset="medium",
        threads=4,
        ffmpeg_params=["-pix_fmt", "yuv420p"]
    )

if __name__ == "__main__":
    gifs = ["1.gif", "2.gif"]  # your files
    stack_with_wipe_transitions(
        gifs,
        transition_duration=0.7,
        speed_factor=2.0,
        output_path="combined.mp4",
        fps=30
    )
    print("Done! Check combined.mp4")


t:   1%|          | 4/536 [00:08<19:21,  2.18s/it, now=None]

Moviepy - Building video combined.mp4.
Moviepy - Writing video combined.mp4



t:   1%|          | 4/536 [00:12<26:45,  3.02s/it, now=None]

Moviepy - Done !
Moviepy - video ready combined.mp4
Done! Check combined.mp4
